### Predict Home

### Predict Away

### Test

In [ ]:
df_all_leagues = pd.read_csv('../data/csv/Leagues.csv', index_col=0)

def get_last_game_mu():
    df_premier_leagues = df_all_leagues[df_all_leagues['league'].str.contains('Premier League')]
    df_mu_last_game = df_premier_leagues[
        (df_premier_leagues['home_team'].str.contains('Manchester United')) & 
        (df_premier_leagues['away_team'].str.contains('Leicester'))].copy()
    df_mu_last_game = df_mu_last_game[df_mu_last_game['date'].str.contains('2024')].copy()
    #df_mu_last_game['date'] = pd.to_datetime(df_mu_last_game['date'], format='%A %B %d, %Y')
    #df_mu_last_game = df_mu_last_game.sort_values('date', ascending=False).head(1)
    
    return df_mu_last_game

df_mu_last_game = get_last_game_mu()
df_mu_last_game.drop(['league', 'matchweek', 'home_team_id', 'away_team_id', 'home_team', 'away_team', 'home_goals', 'away_goals'], axis=1, inplace=True)

X_score = score_dict_vectorizer.transform(df_mu_last_game.copy().to_dict(orient='records'))
mu_result = result_model.predict(X_score)
df_mu_last_game['predicted_result'] = mu_result

X_home_goal = home_dict_vectorizer.transform(df_mu_last_game.copy().to_dict(orient='records'))
X_away_goal = away_dict_vectorizer.transform(df_mu_last_game.copy().to_dict(orient='records'))

mu_home_goals = home_model.predict(X_home_goal)
mu_away_goals = away_model.predict(X_away_goal)


#df_leagues['result'] = [0 if x == 0 else 1 if x > 0 else 2 for x in df_leagues['home_goals'] - df_leagues['away_goals']]
#df_leagues['home_goals'] = y_pred_home_goals
#df_leagues['away_goals'] = y_pred_away_goals

df_mu_last_game = get_last_game_mu()
print(f'Manchester United vs {df_mu_last_game["away_team"].values[0]} -> {mu_home_goals} - {mu_away_goals} | {mu_result} | {df_mu_last_game["home_goals"].values[0]} - {df_mu_last_game["away_goals"].values[0]}')

df_mu_last_game

Manchester United vs Leicester City -> [[0 1 0 0]] - [[1 0 0 0]] | [1] | 3 - 0


,date,league,matchweek,home_team_id,home_team,home_manager,home_formation,home_goals,home_xg,away_xg,...,away_team_form,away_team_league_pos,away_team_points_diff,away_team_away_form,away_team_away_league_pos,away_team_away_points_diff,away_team_average_goals_form,away_team_average_xg_form,away_team_average_goals_form_against,away_team_average_xg_form_against
88,"Sunday November 10, 2024",Premier League,11,19538871,Manchester United,Ruud van Nistelrooy,4-2-3-1,3,0.8,0.6,...,LWWLD,15.0,4,LWD,10.0,-1,2.0,1.633333,2.333333,2.5


In [ ]:
#Predict away goals
df_train_away, df_test_away, y_train_away, y_test_away = get_train_test_split(df_leagues, 'away_goals')
X_train_away, X_test_away, away_dict_vectorizer = get_X_train_test(df_train_away, df_test_away)

y_train_away_no_goal = (y_train_away == 0).astype(int)
y_train_away_one_goal = (y_train_away >= 1).astype(int)
y_train_away_two_goal = (y_train_away >= 2).astype(int)
y_train_away_three_goal = (y_train_away >= 3).astype(int)

y_test_away_no_goal = (y_test_away == 0).astype(int)
y_test_away_one_goal = (y_test_away >= 1).astype(int)
y_test_away_two_goal = (y_test_away >= 2).astype(int)
y_test_away_three_goal = (y_test_away >= 3).astype(int)

y_train_away_combined = np.column_stack([y_train_away_no_goal, y_train_away_one_goal, y_train_away_two_goal, y_train_away_three_goal])
y_test_away_combined = np.column_stack([y_test_away_no_goal, y_test_away_one_goal, y_test_away_two_goal, y_test_away_three_goal])

a_model = MultiOutputClassifier(XGBClassifier(random_state=42))

param_grid = {
    "estimator__max_depth": range(5,60, 5),
    "estimator__learning_rate": [0.01, 0.1],
    "estimator__n_estimators": [200],
    "estimator__subsample": [0.8],
    "estimator__colsample_bytree": [0.8],
}

grid_search = GridSearchCV(
    estimator = a_model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=2,
    n_jobs=-1
)

grid_search.fit(X_train_away, y_train_away_combined)


print('---------------------------------------------------------------------------')
print("Meilleurs hyperparamètres :", grid_search.best_params_)
print("Meilleur score :", grid_search.best_score_)

away_model = grid_search.best_estimator_

Fitting 5 folds for each of 22 candidates, totalling 110 fits
[CV] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.01, estimator__max_depth=5, estimator__n_estimators=200, estimator__subsample=0.8; total time=   4.1s
[CV] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.01, estimator__max_depth=5, estimator__n_estimators=200, estimator__subsample=0.8; total time=   4.2s
[CV] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.01, estimator__max_depth=5, estimator__n_estimators=200, estimator__subsample=0.8; total time=   4.3s
[CV] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.01, estimator__max_depth=5, estimator__n_estimators=200, estimator__subsample=0.8; total time=   4.3s
[CV] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.01, estimator__max_depth=5, estimator__n_estimators=200, estimator__subsample=0.8; total time=   4.4s
[CV] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.01, estimator_

/Users/florian/Workspace/Learning/ml_zoomcamp/.venv/lib/python3.13/site-packages/joblib/externals/loky/process_executor.py:752: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.



[CV] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.1, estimator__max_depth=5, estimator__n_estimators=200, estimator__subsample=0.8; total time=   3.4s
[CV] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.1, estimator__max_depth=5, estimator__n_estimators=200, estimator__subsample=0.8; total time=   3.6s
[CV] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.1, estimator__max_depth=5, estimator__n_estimators=200, estimator__subsample=0.8; total time=   3.6s
[CV] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.01, estimator__max_depth=55, estimator__n_estimators=200, estimator__subsample=0.8; total time=  11.0s
[CV] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.01, estimator__max_depth=55, estimator__n_estimators=200, estimator__subsample=0.8; total time=  10.6s
[CV] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.1, estimator__max_depth=10, estimator__n_estimators=200, estimator__subsample

In [ ]:
y_pred_away = away_model.predict(X_test_away)

accuracy_zero = accuracy_score(y_test_away_no_goal, y_pred_away[:,0])
accuracy_one = accuracy_score(y_test_away_one_goal, y_pred_away[:,1])
accuracy_two = accuracy_score(y_test_away_two_goal, y_pred_away[:,2])
accuracy_three = accuracy_score(y_test_away_three_goal, y_pred_away[:,3])

print(f'Accuracy: 0 Goal -> {accuracy_zero} | 1 Goals -> {accuracy_one} | 2 Goals -> {accuracy_two} | 3 Goals -> {accuracy_three}')

Accuracy: 0 Goal -> 0.725201072386059 | 1 Goals -> 0.725201072386059 | 2 Goals -> 0.7506702412868632 | 3 Goals -> 0.8833780160857909


In [ ]:
df_train_home, df_test_home, y_train_home, y_test_home = get_train_test_split(df_leagues, 'home_goals')
X_train_home, X_test_home, home_dict_vectorizer = get_X_train_test(df_train_home, df_test_home)

y_train_home_no_goal = (y_train_home == 0).astype(int)
y_train_home_one_goal = (y_train_home >= 1).astype(int)
y_train_home_two_goal = (y_train_home >= 2).astype(int)
y_train_home_three_goal = (y_train_home >= 3).astype(int)

y_test_home_no_goal = (y_test_home == 0).astype(int)
y_test_home_one_goal = (y_test_home >= 1).astype(int)
y_test_home_two_goal = (y_test_home >= 2).astype(int)
y_test_home_three_goal = (y_test_home >= 3).astype(int)

y_train_home_combined = np.column_stack([y_train_home_no_goal, y_train_home_one_goal, y_train_home_two_goal, y_train_home_three_goal])
y_test_home_combined = np.column_stack([y_test_home_no_goal, y_test_home_one_goal, y_test_home_two_goal, y_test_home_three_goal])

h_model = MultiOutputClassifier(XGBClassifier(random_state=42))

param_grid = {
    "estimator__max_depth": range(5,60, 5),
    "estimator__learning_rate": [0.01, 0.1],
    "estimator__n_estimators": [200],
    "estimator__subsample": [0.8],
    "estimator__colsample_bytree": [0.8],
}

grid_search = GridSearchCV(
    estimator = h_model,
    param_grid=param_grid,
    scoring='accuracy',
    cv=5,
    verbose=2,
    n_jobs=-1
)

grid_search.fit(X_train_home, y_train_home_combined)


print('---------------------------------------------------------------------------')
print("Meilleurs hyperparamètres :", grid_search.best_params_)
print("Meilleur score :", grid_search.best_score_)

home_model = grid_search.best_estimator_

Fitting 5 folds for each of 22 candidates, totalling 110 fits
[CV] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.01, estimator__max_depth=5, estimator__n_estimators=200, estimator__subsample=0.8; total time=   4.4s
[CV] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.01, estimator__max_depth=5, estimator__n_estimators=200, estimator__subsample=0.8; total time=   4.5s
[CV] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.01, estimator__max_depth=5, estimator__n_estimators=200, estimator__subsample=0.8; total time=   4.5s
[CV] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.01, estimator__max_depth=5, estimator__n_estimators=200, estimator__subsample=0.8; total time=   4.5s
[CV] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.01, estimator__max_depth=5, estimator__n_estimators=200, estimator__subsample=0.8; total time=   4.6s
[CV] END estimator__colsample_bytree=0.8, estimator__learning_rate=0.01, estimator_

In [ ]:
y_pred_home = home_model.predict(X_test_home)

accuracy_zero = accuracy_score(y_test_home_no_goal, y_pred_home[:,0])
accuracy_one = accuracy_score(y_test_home_one_goal, y_pred_home[:,1])
accuracy_two = accuracy_score(y_test_home_two_goal, y_pred_home[:,2])
accuracy_three = accuracy_score(y_test_home_three_goal, y_pred_home[:,3])

print(f'Accuracy: 0 Goal -> {accuracy_zero} | 1 Goals -> {accuracy_one} | 2 Goals -> {accuracy_two} | 3 Goals -> {accuracy_three}')

Accuracy: 0 Goal -> 0.789544235924933 | 1 Goals -> 0.789544235924933 | 2 Goals -> 0.7493297587131368 | 3 Goals -> 0.8257372654155496
